# add report to output for better validation

In [11]:
import pandas as pd
import json

path = '../resource/normalized_entities_gpt_103.csv'
df = pd.read_csv(path)

path = '../../resource/all_unique_entities.json'
entities = json.load(open(path))

for i, row in df.iterrows():
    # find report for each entity from the reports
    entity_name = row['name']
    row['report'] = entities[entity_name]['reports'][0]

    print(row['report'])
    break

    


{'p18/p18001816/s54309228.txt': {'start_ix': 38, 'end_ix': 38, 'text': 'FINAL REPORT INDICATION : Chest pain , dyspnea . COMPARISON : None available . FINDINGS : Chest , PA and lateral . Lung volumes are low . The hilar and cardiomediastinal contours are within normal limits . No chf , focal infiltrate , effusion or pneumothorax is detected . IMPRESSION : No acute pulmonary process identified .'}}


# Analysis

In [5]:
import pandas as pd

data = pd.read_excel('output.xlsx')


# data2['name'] = data['name']

# data2.to_excel('reviewed.xlsx', index=False)

# count data[ui] is nan
count = 0
for i in range(len(data)):
    if pd.isna(data['ui'][i]):
        count += 1


print("count data[ui] is nan: ", count)

data2 = pd.read_excel('reviewed.xlsx')
count = 0
for i in range(len(data)):
    if pd.isna(data['ui'][i]):
        count += 1


print("count data[ui] is nan: ", count)

count data[ui] is nan:  364
count data[ui] is nan:  364


In [4]:
import pandas as pd

data = pd.read_excel('output.xlsx')

data_all = pd.read_excel('reviewed.xlsx')

total_rows = len(data_all)
different_rows = 0
na2ui = 0
ui2na = 0
ui2ui = 0
none_rows = 0
same_rows = 0

for i, row in data_all.iterrows():
    # now we use data200
    ui1 = data.iloc[i]['ui']
    ui2 = row['ui']
    
    # Check if both values are not NaN and they are different
    if not (pd.isna(ui1) and pd.isna(ui2)) and ui1 != ui2:
        # print(f"Row {i} has different 'ui': {row['name']}")
        # print(f"  data: {ui1}")
        # print(f"  data_reviewed: {ui2}")
        # print()
        different_rows += 1

        # Further categorize the differences
        if pd.isna(ui1):
            # print(f"  data 'ui' is NaN, but data_reviewed 'ui' is {ui2}")
            na2ui += 1
        elif pd.isna(ui2):
            # print(f" {i} data {row['name']}'ui' is {ui1}, but data_reviewed 'ui' is NaN")
            ui2na += 1
        else:
            print(f" {i} data {row['name']} 'ui' is {ui1}, and data_reviewed 'ui' is {ui2}")
            ui2ui += 1

    # both values are NaN
    elif pd.isna(ui1) and pd.isna(ui2):
        # print(f"Row {i} has same 'ui': {row['name']}")
        # print(f"  data: {ui1}, { data.iloc[i]['name']}")
        # print(f"  data_reviewed: {ui2}, {row['name']}")
        none_rows += 1

    # both values are same
    elif ui1 == ui2:
        same_rows += 1

print(f"Total number of rows : {total_rows}")
print(f"Total number of rows was changing: {different_rows}")
print(f"Total number of rows na2ui: {na2ui}")
print(f"Total number of rows ui2na: {ui2na}")
print(f"Total number of rows ui2ui: {ui2ui}")
print(f"Total number of rows none: {none_rows}")
print(f"Total number of rows same: {same_rows}")

 24 data Pulmonary 'ui' is C0024109, and data_reviewed 'ui' is C2709248
 63 data little'ui' is C1440917, but data_reviewed 'ui' is NaN
 97 data cardiac 'ui' is C0018787, and data_reviewed 'ui' is C1522601
 241 data Cardiac 'ui' is C0018787, and data_reviewed 'ui' is C1522601
 449 data severity 'ui' is C0521117, and data_reviewed 'ui' is C0439793 
 478 data worrisome 'ui' is C0233481, and data_reviewed 'ui' is C5447511 
 480 data constant 'ui' is C1547014, and data_reviewed 'ui' is C1948059 
 502 data OPACITIES 'ui' is C1265876, and data_reviewed 'ui' is C0029053
 766 data internal jugular central venous 'ui' is C0398279, and data_reviewed 'ui' is C2202213
 840 data ARDS 'ui' is C0035222, and data_reviewed 'ui' is C2887484 
 871 data Little'ui' is C0023882, but data_reviewed 'ui' is NaN
 883 data cardiomegally 'ui' is C0007226, and data_reviewed 'ui' is C0018800 
 959 data mm 'ui' is C0456680, and data_reviewed 'ui' is C0439200
 983 data Opacity 'ui' is C1265876, and data_reviewed 'ui' 

a) 方法链接CUI，human标注nan：
例子：
Row 63: little (data: C1440917, data_reviewed: nan)   
Row 871: Little (data: C0023882, data_reviewed: C0700321)   
特点：
这些词是描述性的或修饰性的，"little"
方法可能过于积极地为这些词分配了 CUI，而人工标注者可能认为这些词在上下文中不需要特定的医学编码。



b) 方法与human标注的CUI不一致：
"Heart" (Row 17): C0018787 -> C1522601  
"Pulmonary" (Row 24): C0024109 -> C2709248  
"cardiac" (Row 97): C0018787 -> C1522601  
"severity" (Row 449): C0521117 -> C0439793  
"OPACITIES" (Row 502): C1265876 -> C0029053  

特点：

解剖学术语（"Heart", "Pulmonary"）
形容词（"cardiac"）
抽象概念（"severity"）
医学症状描述（"OPACITIES"）

这种情况可能反映了UMLS中概念的细微差别，或者人工审核者根据上下文选择了更精确的CUI。



c) 方法标注nan，human标注CUI：

"surfaces" (Row 7): nan -> C0205148  
"positioning" (Row 50): nan -> C0733755  
"lower" (Row 134): nan -> C0441994  
"compressive" (Row 169): nan -> C0376507  
"well" (Row 181): nan -> C3146287  

特点：
解剖位置描述（"surfaces", "lower"）
医疗操作相关词（"positioning"）
症状描述（"compressive"）
常见词汇在医学上下文中的特殊含义（"well"）


d)人工标注和方法都是nan的情况（152个例子，列举一些典型的）：

 in standard position, tips, 3.5 cm, components, 
  not as well seen on this study, 4 cm above, deeper, 1-2 mm, A, rod, overlies, T 5 through T 9, of T 5 through T 9, intrafissural, 1.5 cm
 
特点：

包含了大量的描述性术语和短语，这些可能太具体或上下文相关，难以映射到标准化的医学概念。  
许多是解剖位置的描述或医疗设备的位置描述。  
有些是相对普通的词，没有什么医学特殊含义。  
包含了一些非常具体的测量值（如 "5 cm above", "3.5 cm"）。  
有一些是一般性的描述词（如 "partially", "minimally", "essentially"），在医学上下文中可能有特殊含义，但不易标准化。  
包含了一些医学缩写（如 "AP", "IJ"）和专有名词（如 "Swan"）。  
有一些复合词或短语（如 "well - expanded", "not well seen"）可能难以映射到单一的标准化概念。

# Clean

In [ ]:
import pandas as pd

data = pd.read_excel('output.xlsx')

data2 = pd.read_excel('reviewed.xlsx')

# data2['name'] = data['name']

# data2.to_excel('reviewed.xlsx', index=False)

# check data[name] and data2[name] are same
for i, row in data2.iterrows():
    if data.iloc[i]['name'] != row['name']:
        print(f"Row {i} has different 'name': {row['name']}")
        print(f"  data: {data.iloc[i]['name']}")
        print(f"  data2: {row['name']}")
        print()



In [4]:
import pandas as pd
data2 = pd.read_excel('output.xlsx')


# write first 3 columns of data2 to .csv
data2[['name', 'ui', 'normalized_name']].to_csv('output.csv', index=False)